In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):    #@save
    """计算二维互相关运算"""
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

X=torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [2]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

边缘检测

In [3]:
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [4]:
K=torch.tensor([[1.0,-1.0]])
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [5]:
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习卷积核

In [6]:
# 构造一个二维卷积层,它具有1个输出通道和形状为(1,2)的卷积核
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式(批量大小,通道,高度,宽度),其中批量大小和通道数都为1
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr=3e-2

for i in range(10):
    Y_hat=conv2d(X)
    l=(Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data-=lr*conv2d.weight.grad   # 迭代卷积核
    if (i+1)%2==0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 5.070
epoch 4, loss 1.634
epoch 6, loss 0.595
epoch 8, loss 0.231
epoch 10, loss 0.093


In [7]:
conv2d.weight.data.reshape((1,2))

tensor([[ 1.0242, -0.9618]])

Exercises

In [8]:
Z=torch.ones(6,6)-torch.eye(6)
Z

tensor([[0., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 1.],
        [1., 1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 0.]])

In [9]:
K=torch.tensor([[1.0,-1.0]])
corr2d(Z,K)

tensor([[-1.,  0.,  0.,  0.,  0.],
        [ 1., -1.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  1., -1.],
        [ 0.,  0.,  0.,  0.,  1.]])

In [10]:
corr2d(Z,K.t())

tensor([[-1.,  1.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.]])

In [11]:
def train(conv2d,X,Y,lr,num_epochs):
    for i in range(num_epochs):
        Y_hat=conv2d(X)
        l=(Y_hat-Y)**2
        conv2d.zero_grad()
        l.sum().backward()
        conv2d.weight.data-=lr*conv2d.weight.grad
        if (i+1)%2==0:
            print(f'epoch {i+1}, loss {l.sum():.3f}')
    return conv2d.weight.data.reshape((1,2))

# nn.Conv2d的输入输出格式是四维,而自定义的Conv2D只有二维,自动求导时会报错
conv2d=Conv2D(kernel_size=(1,2))

try:
    train(conv2d,X,Y,lr,num_epochs=10)
except Exception as e:
    print(e,'\n')

X=X.reshape((6,8))  # 将输入的形状改为二维
Y=Y.reshape((6,7))
train(conv2d,X,Y,lr,num_epochs=10)

The size of tensor a (0) must match the size of tensor b (7) at non-singleton dimension 3 

epoch 2, loss 14.720
epoch 4, loss 5.510
epoch 6, loss 2.170
epoch 8, loss 0.874
epoch 10, loss 0.356


tensor([[ 1.0538, -0.9309]])